### AI Agent Tool Call Testing

In [ ]:
from deepeval.tracing import observe, update_current_span

#### AI Agent with Tools

In [88]:
from langchain.tools import tool
from deepeval.test_case import ToolCall
from langchain_community.tools import DuckDuckGoSearchRun

s_tool = DuckDuckGoSearchRun()

@tool
@observe(type="tool")
def search_tool(query: str):
    "Tool to search online for given query and return result"
    return s_tool.run(query)

@tool
@observe(type="tool")
def add_numbers(a: int, b: int):
    "Add two numbers and returns result"
    return int(a) + int(b)

@tool
@observe(type="tool")
def subtract_numbers(a: int, b: int):
    "Subtract two numbers and returns result"
    return int(a) - int(b)

tools = [add_numbers, subtract_numbers, search_tool]
AVAILABLE_TOOLS = [
    ToolCall(name=t.name, input_parameters={})
    for t in tools
]

#### Local LLMs

In [ ]:
from langchain_ollama import ChatOllama
@observe(type="llm",
         name="Local LLM",
         model="qwen2.5:latest")
def local_llm():
    llm = ChatOllama(base_url="http://localhost:11434", 
                      model="qwen2.5:latest", 
                      temperature=0.5, 
                      max_token=250)
    return llm.bind_tools(tools=tools)
llm = local_llm()

In [ ]:
from deepeval.models import OllamaModel
local_llm_deepseek = OllamaModel(
    model="deepseek-r1:8b",
    base_url="http://localhost:11434",
    temperature=0.5,
    generation_kwargs={"num_predict": 500}
)

#### Create ChatPrompt, agent

In [ ]:
from langchain_classic.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from deepeval.test_case import ToolCall

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant with access to tools."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])
@observe(type="agent", 
         available_tools=["add_numbers", "subtract_numbers", "search_tool"])
def main_ai_agent(query):
    agent = create_tool_calling_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent,
                                   tools=tools,
                                   verbose=True,
                                   handle_parsing_errors=True,
                                   return_intermediate_steps=True
    )
    response = agent_executor.invoke({"input": query, "chat_history": []})
    return response

#### Extract Tools called from Response

In [ ]:
from langchain_core.agents import AgentAction
from typing import List
def extract_tools_from_response(response: dict[str: any]) -> List[ToolCall]:
    deepeval_tools: List[ToolCall] = []
    intermediate_steps = response.get("intermediate_steps", [])
    for action, _ in intermediate_steps:
        if isinstance(action, AgentAction):
            tool_name = action.tool
            tool_args = action.tool_input
            deepeval_tools.append(
                ToolCall(name=tool_name, 
                         input_parameters=tool_args)
            ) 
    return deepeval_tools

In [ ]:
search_tool_response = main_ai_agent("Tell me a fun fact about giraffes.")
extract_tools_from_response(search_tool_response)

#### Evaluation (Tool Correctness Metric)

In [ ]:
from deepeval.dataset import Golden
tool_correctness_test_data = [
    {
        "input": "What is 15 + 20?",
        "expected_tool_calls": [
            ToolCall(name="add_numbers", input_parameters={"a": 15, "b": 20})
        ]
    },
    {
        "input": "What is the highest mountain in the world?",
        "expected_tool_calls": [
            ToolCall(name="search_tool", 
                     input_parameters={"query": "highest mountain in the world"})
        ]
    },
    {
        "input": "Calculate the difference between 500 and 150?",
        "expected_tool_calls": [
            ToolCall(name="subtract_numbers", 
                     input_parameters={"a": 500, "b": 150})
        ]
    },
    {
        "input": "Tell me a fun fact about giraffes.",
        "expected_tool_calls": [
            ToolCall(name="search_tool", 
                     input_parameters={"query": "fun fact about giraffes"})
        ]
    }
]

In [ ]:
goldens_data = []
for test_data in tool_correctness_test_data:
    golden = Golden(
        input= test_data["input"],
        expected_tools=test_data["expected_tool_calls"]
    )
    goldens_data.append(golden)

In [89]:
from deepeval.test_case import LLMTestCase
from deepeval.dataset import EvaluationDataset
from deepeval.evaluate import evaluate
from deepeval.metrics import ToolCorrectnessMetric

data_set = EvaluationDataset(goldens=goldens_data)
processed_testcases:List[LLMTestCase] = []
for golden in data_set.goldens:
    response = main_ai_agent(golden.input)
    llm_testcase = LLMTestCase(
        input=golden.input,
        expected_tools=golden.expected_tools,
        tools_called=extract_tools_from_response(response)
    )
    processed_testcases.append(llm_testcase)
evaluate(processed_testcases, 
         metrics=[ToolCorrectnessMetric(model=local_llm_deepseek, 
                                        threshold=0.5, 
                                        available_tools=AVAILABLE_TOOLS)])




> Entering new AgentExecutor chain...

Invoking: `add_numbers` with `{'a': 15, 'b': 20}`


35The result of 15 + 20 is 35.

> Finished chain.
Full Response: {'input': 'What is 15 + 20?', 'chat_history': [], 'output': 'The result of 15 + 20 is 35.', 'intermediate_steps': [(ToolAgentAction(tool='add_numbers', tool_input={'a': 15, 'b': 20}, log="\nInvoking: `add_numbers` with `{'a': 15, 'b': 20}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:latest', 'created_at': '2025-12-05T17:33:20.274984Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3374990959, 'load_duration': 95396542, 'prompt_eval_count': 264, 'prompt_eval_duration': 1653555875, 'eval_count': 28, 'eval_duration': 1414231957, 'logprobs': None, 'model_name': 'qwen2.5:latest', 'model_provider': 'ollama'}, id='lc_run--f5c9045c-1d4d-49f4-8325-4ff0391e705f', tool_calls=[{'name': 'add_numbers', 'args': {'a': 15, 'b': 20}, 'id': '45db0ee9-e12e-413f-9b70-1f128491754e'

[Confident AI Trace Log]  Successfully posted trace (0 traces remaining in queue, 1 in flight): 
https://app.confident-ai.com/project/cmiou0pbn01p0qh1fbxhly3fa/observatory/traces/93a001bb-c84b-4360-a9b8-72325ecb8
6e1 
To disable dev logging, set CONFIDENT_TRACE_VERBOSE=0 as an environment variable.

The highest mountain in the world is Mount Everest. It has the highest elevation above sea level at approximately 8,849 meters (29,032 feet). However, if you consider the total height from its base below sea level to its peak, Mauna Kea in Hawaii stands taller.

> Finished chain.
Full Response: {'input': 'What is the highest mountain in the world?', 'chat_history': [], 'output': 'The highest mountain in the world is Mount Everest. It has the highest elevation above sea level at approximately 8,849 meters (29,032 feet). However, if you consider the total height from its base below sea level to its peak, Mauna Kea in Hawaii stands taller.', 'intermediate_steps': [(ToolAgentAction(tool='search_tool', tool_input={'query': 'highest mountain in the world'}, log="\nInvoking: `search_tool` with `{'query': 'highest mountain in the world'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:latest', 'created_at': '2025-12-05T17:33:23.16813Z

[Confident AI Trace Log]  Successfully posted trace (0 traces remaining in queue, 1 in flight): 
https://app.confident-ai.com/project/cmiou0pbn01p0qh1fbxhly3fa/observatory/traces/ff7ac644-9ade-4fb3-9af2-56d6ea486
71d 
To disable dev logging, set CONFIDENT_TRACE_VERBOSE=0 as an environment variable.

The difference between 500 and 150 is 350.

> Finished chain.
Full Response: {'input': 'Calculate the difference between 500 and 150?', 'chat_history': [], 'output': 'The difference between 500 and 150 is 350.', 'intermediate_steps': [(ToolAgentAction(tool='subtract_numbers', tool_input={'a': 500, 'b': 150}, log="\nInvoking: `subtract_numbers` with `{'a': 500, 'b': 150}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:latest', 'created_at': '2025-12-05T17:33:32.573232Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2031476292, 'load_duration': 65913917, 'prompt_eval_count': 268, 'prompt_eval_duration': 193143458, 'eval_count': 30, 'eval_duration': 1552831127, 'logprobs': None, 'model_name': 'qwen2.5:latest', 'model_provider': 'ollama'}, id='lc_run--af3b5dd0-059f-4596-935b-18952bc3e35c', tool_calls=[{'name': 'subtract_numbers', 'args': {'a': 500, 'b': 150}, 'id': '3bb4d992-77b1-42e4-9972-5e116d296489', 'type': 'tool_ca

...

[Confident AI Trace Log]  Successfully posted trace (0 traces remaining in queue, 1 in flight): 
https://app.confident-ai.com/project/cmiou0pbn01p0qh1fbxhly3fa/observatory/traces/b40bc245-fe2d-4faa-8239-916b61d97
49c 
To disable dev logging, set CONFIDENT_TRACE_VERBOSE=0 as an environment variable.



Metrics Summary

  - ✅ Tool Correctness (score: 1.0, threshold: 0.5, strict: False, evaluation model: None, reason: [
	 Tool Calling Reason: All expected tools ['search_tool'] were called (order not considered).
	 Tool Selection Reason: The agent correctly selected the search_tool, which is appropriate for retrieving information like fun facts about giraffes. The other tools (add_numbers and subtract_numbers) are irrelevant to the task and were not used, indicating no unnecessary or missing tools were selected.
]
, error: None)

For test case:

  - input: Tell me a fun fact about giraffes.
  - actual output: None
  - expected output: None
  - context: None
  - retrieval context: None


Metrics Summary

  - ✅ Tool Correctness (score: 1.0, threshold: 0.5, strict: False, evaluation model: None, reason: [
	 Tool Calling Reason: All expected tools ['search_tool'] were called (order not considered).
	 Tool Selection Reason: The search_tool was appropriately selected to retrieve factual inf

⚠ WARNING: No hyperparameters logged.
» ]8;id=624456;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Done 🎉! View results on 
]8;id=312733;https://app.confident-ai.com/project/cmiou0pbn01p0qh1fbxhly3fa/test-runs/cmit5c3v500r0my1f0sberb2b/regression-testing\https://app.confident-ai.com/project/cmiou0pbn01p0qh1fbxhly3fa/test-runs/cmit5c3v500r0my1f0sberb2b/regression-testi]8;;\
]8;id=312733;https://app.confident-ai.com/project/cmiou0pbn01p0qh1fbxhly3fa/test-runs/cmit5c3v500r0my1f0sberb2b/regression-testing\ng]8;;\

EvaluationResult(test_results=[TestResult(name='test_case_3', success=True, metrics_data=[MetricData(name='Tool Correctness', threshold=0.5, success=True, score=1.0, reason="[\n\t Tool Calling Reason: All expected tools ['search_tool'] were called (order not considered).\n\t Tool Selection Reason: The agent correctly selected the search_tool, which is appropriate for retrieving information like fun facts about giraffes. The other tools (add_numbers and subtract_numbers) are irrelevant to the task and were not used, indicating no unnecessary or missing tools were selected.\n]\n", strict_mode=False, evaluation_model=None, error=None, evaluation_cost=0.0, verbose_logs='Expected Tools:\n[\n    ToolCall(\n        name="search_tool",\n        input_parameters={\n            "query": "fun fact about giraffes"\n        }\n    )\n] \n \nTools Called:\n[\n    ToolCall(\n        name="search_tool",\n        input_parameters={\n            "query": "fun facts about giraffes"\n        }\n    )\n] \